#C6W6 TAXI SHITKOV

###1. Загрузите обучающие выборки прошлой недели, перечислите используемые в моделях признаки и посчитайте качество прогнозов моделей, настроенных на данных до апреля 2016, в мае 2016.

In [0]:
import numpy as np
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os

In [0]:
from datetime import datetime

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [0]:
#список регионов из C6W2:
#Загрузить агрегированные данные за май 2016:
filepath = '/content/gdrive/My Drive/ml/c6/data/'
filename = '2016-5.csv'
data = pd.read_csv(filepath + filename, index_col = 0)
data_arr = data.to_numpy()
data_mean = np.asarray([data_arr.T[i].mean() for i in range(len(data_arr.T))])

In [0]:
#Список ячеек для удаления из датасета:
rem = np.asarray([-1 if (data_mean > 5)[i] == True else i for i in range(len(data_mean))])
rem[rem >= 0]
rem = data.columns[rem]
rem_list = list(rem)

In [0]:
#Список регионов для работы
ind = np.asarray([i if (data_mean >= 5)[i] == True else 0 for i in range(len(data_mean))])
ind = ind[ind > 0]
ind = data.columns[ind]
cells = list(ind)

In [0]:
#последние N месяцев
import os

N = 6

path = '/content/gdrive/My Drive/ml/c6/data/'
fl = []
for filename in os.listdir(path):
  fl.append(filename)
  
fl.sort()

fl = fl[-(N + 1):-1]

print(fl)

['2016-1.csv', '2016-2.csv', '2016-3.csv', '2016-4.csv', '2016-5.csv', '2016-6.csv']


In [0]:
from datetime import datetime
def make_data(fl, rem, path):  
  #загрузить данные
  month_arr = []
  for m in range(len(fl)):
    filename = str(fl[m])
    data = pd.read_csv(filepath + filename, index_col = 0)
    month_arr.append(pd.DataFrame(data, index = [datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in data.index]))

  data = pd.concat(month_arr)

  data.drop(rem, axis='columns', inplace=True)
  
  return data

data = make_data(fl, rem, path)
data.head()

,region_1075,region_1076,region_1077,region_1125,region_1126,region_1127,region_1128,region_1129,region_1130,region_1131,region_1132,region_1172,region_1173,region_1174,region_1175,region_1176,region_1177,region_1178,region_1179,region_1180,region_1181,region_1182,region_1183,region_1184,region_1221,region_1222,region_1223,region_1224,region_1225,region_1227,region_1228,region_1229,region_1230,region_1231,region_1232,region_1233,region_1234,region_1235,region_1272,region_1273,...,region_1376,region_1377,region_1378,region_1380,region_1382,region_1383,region_1384,region_1385,region_1386,region_1387,region_1388,region_1389,region_1390,region_1426,region_1431,region_1434,region_1435,region_1436,region_1437,region_1438,region_1439,region_1441,region_1442,region_1480,region_1482,region_1483,region_1530,region_1532,region_1533,region_1580,region_1630,region_1684,region_1733,region_1734,region_1783,region_2068,region_2069,region_2118,region_2119,region_2168
2016-01-01 00:00:00,80.0,144.0,50.0,77.0,319.0,402.0,531.0,617.0,846.0,267.0,35.0,16.0,39.0,31.0,29.0,40.0,548.0,649.0,759.0,930.0,695.0,645.0,323.0,24.0,26.0,34.0,25.0,57.0,14.0,420.0,782.0,1001.0,1273.0,938.0,403.0,664.0,519.0,213.0,51.0,49.0,...,55.0,51.0,27.0,23.0,22.0,260.0,908.0,329.0,56.0,93.0,58.0,53.0,72.0,42.0,52.0,5.0,20.0,70.0,66.0,38.0,30.0,30.0,23.0,18.0,42.0,38.0,20.0,25.0,39.0,13.0,12.0,0.0,2.0,44.0,5.0,41.0,4.0,70.0,7.0,66.0
2016-01-01 01:00:00,91.0,211.0,49.0,134.0,404.0,420.0,370.0,453.0,594.0,224.0,51.0,38.0,91.0,71.0,29.0,34.0,588.0,534.0,518.0,671.0,830.0,645.0,338.0,22.0,67.0,88.0,57.0,59.0,11.0,613.0,828.0,1116.0,1514.0,830.0,420.0,567.0,663.0,271.0,93.0,86.0,...,163.0,107.0,69.0,40.0,15.0,312.0,990.0,406.0,91.0,156.0,113.0,64.0,106.0,78.0,51.0,7.0,44.0,127.0,133.0,56.0,45.0,53.0,62.0,37.0,92.0,64.0,53.0,64.0,130.0,23.0,29.0,0.0,5.0,2.0,2.0,4.0,0.0,47.0,1.0,29.0
2016-01-01 02:00:00,90.0,146.0,23.0,110.0,393.0,425.0,313.0,366.0,377.0,138.0,21.0,31.0,75.0,78.0,22.0,56.0,612.0,431.0,401.0,503.0,683.0,530.0,374.0,13.0,82.0,82.0,55.0,68.0,25.0,615.0,763.0,962.0,1363.0,739.0,484.0,705.0,541.0,214.0,100.0,93.0,...,183.0,130.0,101.0,39.0,10.0,235.0,922.0,269.0,61.0,112.0,127.0,78.0,112.0,91.0,50.0,1.0,29.0,101.0,107.0,52.0,54.0,33.0,51.0,67.0,129.0,63.0,57.0,125.0,137.0,35.0,47.0,0.0,3.0,0.0,4.0,0.0,0.0,69.0,1.0,14.0
2016-01-01 03:00:00,32.0,87.0,16.0,62.0,252.0,399.0,324.0,309.0,327.0,166.0,10.0,28.0,56.0,63.0,9.0,56.0,711.0,394.0,391.0,523.0,587.0,715.0,410.0,15.0,49.0,85.0,49.0,51.0,7.0,542.0,782.0,968.0,1202.0,771.0,742.0,615.0,263.0,152.0,52.0,55.0,...,188.0,115.0,79.0,25.0,5.0,152.0,550.0,157.0,51.0,110.0,98.0,63.0,89.0,90.0,43.0,6.0,19.0,59.0,85.0,44.0,42.0,44.0,44.0,47.0,94.0,34.0,55.0,86.0,124.0,34.0,46.0,0.0,2.0,4.0,5.0,1.0,0.0,21.0,0.0,9.0
2016-01-01 04:00:00,24.0,43.0,10.0,53.0,145.0,254.0,264.0,333.0,318.0,145.0,5.0,8.0,22.0,22.0,3.0,47.0,536.0,506.0,388.0,381.0,403.0,550.0,256.0,4.0,29.0,23.0,30.0,53.0,2.0,444.0,703.0,646.0,647.0,560.0,701.0,379.0,150.0,62.0,26.0,29.0,...,135.0,57.0,34.0,24.0,4.0,95.0,351.0,96.0,32.0,56.0,65.0,48.0,74.0,76.0,37.0,0.0,17.0,73.0,70.0,27.0,40.0,41.0,54.0,50.0,83.0,46.0,45.0,81.0,113.0,26.0,43.0,0.0,0.0,1.0,1.0,0.0,0.0,26.0,1.0,6.0


In [0]:
def make_features(data, k_features, k_poly):
  col = []

  for i in range(1, k_features + 1):
    col.append(str('sin_' + str(i)))
    col.append(str('cos_' + str(i)))
  for i in range(1, k_poly + 1):
    col.append(str('poly_' + str(i)))
    
  features = pd.DataFrame(columns = col, index = data.index)
  
  features['num'] = [int(i) for i in range(1, len(data) + 1)]
  
  for k in range(1, k_features + 1):
    features[str('sin_' + str(k))] = [np.sin(features.num[x] * 2 * np.pi * k / 168) for x in range(len(data))]
    features[str('cos_' + str(k))] = [np.cos(features.num[x] * 2 * np.pi * k / 168) for x in range(len(data))]

  for k in range(1, k_poly + 1):
    features[str('poly_' + str(k))] = [features.num[x] ** k  for x in range(len(data))]  

  features['year'] = [data.index[i].year for i in range(len(data))]
  features['month'] = [data.index[i].month for i in range(len(data))]
  features['day'] = [data.index[i].day for i in range(len(data))]
  features['weekday'] = [data.index[i].weekday() for i in range(len(data))]
  features['hour'] = [data.index[i].hour for i in range(len(data))]
  features.drop(['num'], axis='columns', inplace=True)
  
  return features

In [0]:
#Количество создаваемых признаков sin, cos, poly
k_features = 10
k_poly = 2

fl = []
filepath = '/content/gdrive/My Drive/ml/c6/c6w6/regions2/'
for filename in os.listdir(filepath):
  fl.append(filename)
if 'features.csv' not in fl:
  print('features calculaing: WORK')
  features = make_features(data, k_features, k_poly)
  features.to_csv('/content/gdrive/My Drive/ml/c6/c6w6/regions2/' + 'features.csv')
  print('features calculaing: DONE')
else:
  features = pd.read_csv('/content/gdrive/My Drive/ml/c6/c6w6/regions2/' + 'features.csv', index_col = 0)
  print('features calculaing: DONE')

features calculaing: DONE


In [0]:
fl = []
filepath = '/content/gdrive/My Drive/ml/c6/c6w6/regions2/'
for filename in os.listdir(filepath):
  reg = 'region_' + filename[:-4]
  fl.append(reg)

K  = 23
Kd = 7

sl = [12, 24, 168, 672]

df = []
for i in range(len(cells)):
  #создать датафрейм по региону
  df.append(pd.DataFrame(data[cells[i]]))
  #проверка наличия датасета
  if cells[i] not in fl:
    print(str(cells[i]) + ': WORK')
    #переименовать столбец
    df[i].columns = ['yT']
    #добавить значение количества поездок за предыдущие K часов
    for j in range(1, K + 1):
      df[i][str('yT-' + str(j))] = df[i][df[i].columns[0]].shift(j)
    #добавить значение количества поездок за предыдущие dK часов * 24
    for j in range(1, Kd + 1):
      df[i][str('yT-' + str(j * 24))] = df[i][df[i].columns[0]].shift(24 * j)
    #добавить сумму поездок за предыдущие sl[i] часов
    for j in sl:
      df[i]['sum_' + str(j)] = [np.nan for l in range(j)] + [sum([df[i][df[i].columns[0]][n] for n in range(k, k + j)]) for k in range(len(df[i]) - j)]
    #склеить текущий датасет с features
    df[i] = pd.concat([df[i], features], axis=1)
    #добавить отклик на +1...+6 часов
    for j in range(1, 7):
      df[i][str('yT+' + str(j))] = df[i][df[i].columns[0]].shift(-j)
    #сохранить данные на диск
    df[i].to_csv('/content/gdrive/My Drive/ml/c6/c6w6/regions2/' + str(cells[i][-4:]) + '.csv')
    print(str(cells[i]) + ': DONE')

In [0]:
#считать данные
filepath = '/content/gdrive/My Drive/ml/c6/c6w6/regions2/'
fl = []
for filename in os.listdir(filepath):
  fl.append(filename)
df = []  
for i in range(len(fl)):
  df.append(pd.read_csv(filepath + fl[i], index_col = 0))

#склеить датафреймы  
data = pd.concat(df, sort=False)

In [0]:
#заменить признак weekday по OneHot
data = pd.concat([data, pd.get_dummies(data.weekday, prefix = 'weekday')], axis = 1)
#удалить столбец "weekday"
data.drop(['weekday'], axis='columns', inplace=True)

In [0]:
#заменить признак hour по OneHot
data = pd.concat([data, pd.get_dummies(data.hour, prefix = 'hour')], axis = 1)
#удалить столбец "hour"
data.drop(['hour'], axis='columns', inplace=True)

In [0]:
#Обучающая выборка: train_data
data_train = data[data.index < '2016-04-30 18:00:00'].dropna()
#Тестовая выборка : test_data
data_test = data[(data.index < '2016-05-31 18:00:00') & (data.index >= '2016-04-30 23:00:00')].dropna()

In [0]:
#целевые метки
target_labels = []
for i in range(1, 7):
  target_labels.append(str('yT+' + str(i)))

In [0]:
y_time   = ['year','month','day','yT','yT-1','yT-2','yT-3','yT-4','yT-5','yT-6','yT-7','yT-8','yT-9','yT-10','yT-11','yT-12','yT-13','yT-14','yT-15','yT-16','yT-17']
hour_col = ['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23']

In [0]:
X_train = data_train.copy()
X_train.drop(target_labels + y_time + hour_col, axis='columns', inplace=True)
#удалить столбец "region"
X_train.drop(['region'], axis='columns', inplace=True)

X_test = data_test.copy()
X_test.drop(target_labels + y_time + hour_col, axis='columns', inplace=True)
#удалить столбец "region"
X_test.drop(['region'], axis='columns', inplace=True)

In [0]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [0]:
%%time
from sklearn.linear_model import Lasso

alph = [0.5, 0.5, 0.5, 0.3, 0.1, 0.1]

lasso = []
err   = []
regr  = []


for i in range(6):
  y_train = pd.DataFrame(data_train[target_labels[i]]).values
  y_test  = pd.DataFrame(data_test[target_labels[i]]).values
  
  lasso.append(Lasso(alpha = alph[i]))
  lasso[i].fit(X_train_scaled, y_train)
  y_test_pred = lasso[i].predict(scaler.transform(X_test))
  y_test_pred = y_test_pred * (y_test_pred > 0)
  current_err = sum(abs(data_test[target_labels[i]].values - y_test_pred))/len(y_test_pred)
  print(i, alph[i], current_err)
  err.append(current_err)
  regr.append(lasso[i])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.


(0, 0.5, 28.308497962551446)
(1, 0.5, 32.352915349354085)
(2, 0.5, 34.5552609314065)
(3, 0.3, 35.52358727495987)
(4, 0.1, 35.6679210543814)
(5, 0.1, 36.169326433970376)
CPU times: user 2min 8s, sys: 1.59 s, total: 2min 10s
Wall time: 2min 8s


In [0]:
for i in range(len(regr)):
  print(zip(X_test.columns[regr[i].coef_ > 0], regr[i].coef_[regr[i].coef_ > 0]))

[('yT-19', 1.7142521110527695), ('yT-22', 45.101749962185146), ('yT-23', 101.53038920033748), ('yT-144', 25.99482148739954), ('yT-168', 136.1484319896912), ('sum_672', 0.622083897750811), ('cos_1', 0.6013787496760077), ('sin_2', 3.240660955819526), ('sin_4', 2.772081635743124), ('cos_4', 2.2854015630728997), ('cos_6', 1.3080654911073073), ('cos_8', 1.8913484684892745), ('sin_9', 4.429076257109779), ('cos_9', 0.18341072794274224), ('weekday_3', 0.5678314783385225)]
[('yT-21', 28.679697070129414), ('yT-22', 150.81545746401545), ('yT-23', 13.559064341764437), ('yT-144', 11.372562739603753), ('yT-168', 108.03917630936662), ('sum_672', 31.758735600939403), ('sin_1', 0.23868366172613925), ('sin_2', 0.4705260254024182), ('cos_3', 1.69824360220985), ('sin_4', 3.0935098513849413), ('cos_4', 1.1411721128990127), ('cos_6', 3.1303925582022822), ('cos_8', 1.7951251915014665), ('sin_9', 4.72065661037411), ('cos_9', 2.9545595602412216), ('weekday_4', 1.7397074058637356)]
[('yT-20', 29.479330831233007

In [0]:
#Ошибка за май
print(sum(err)/6)

33.762918167770614


###2. Попробуйте добавить признаки. Обучайте обновлённые модели на данных до апреля 2016 включительно и считайте качество новых прогнозов на мае. Если добавляемый признак не улучшает качество, всё равно оставьте доказательства этому в ноутбуке.

Признаки из сырых данных:

* средняя длительность поездок(расчет)
* среднее количество пассажиров(passenger_count)
* среднее расстояние по счётчику(trip_distance)
* доли поездок, совершаемых по тарифам каждого из типов(RateCodeID)
* доли способов оплаты поездок(Payment_type)
* средняя стоимость поездок(fare_amount)
* доли провайдеров данных(VendorID)
* Количество поездок в текущую географическую зону

In [0]:
#список регионов из C6W2:
#Загрузить агрегированные данные за май 2016:
filepath = '/content/gdrive/My Drive/ml/c6/data/'
filename = '2016-5.csv'
data = pd.read_csv(filepath + filename, index_col = 0)
data_arr = data.to_numpy()
data_mean = np.asarray([data_arr.T[i].mean() for i in range(len(data_arr.T))])

In [0]:
#Список ячеек для удаления из датасета:
rem = np.asarray([-1 if (data_mean > 5)[i] == True else i for i in range(len(data_mean))])
rem[rem >= 0]
rem = data.columns[rem]
rem_list = list(rem)

In [0]:
#Список регионов для работы
ind = np.asarray([i if (data_mean >= 5)[i] == True else 0 for i in range(len(data_mean))])
ind = ind[ind > 0]
ind = data.columns[ind]
cells = list(ind)

In [0]:
from datetime import datetime
import calendar
from scipy.stats import binned_statistic_2d
from datetime import timedelta

In [0]:
addr = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_"
month_list = ['2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06']

In [0]:
regions = pd.read_csv('/content/gdrive/My Drive/ml/c6/data/' + 'regions.csv', ';')

#вытащить уникальные границы из regions.csv
xbins = regions.west.unique()
xbins = np.append(xbins, -73.70001)
ybins = regions.south.unique()
ybins = np.append(ybins, 40.91553)

In [0]:
ind_list = [int(ind[i][-4:]) for i in range(len(ind))]

In [0]:
#почистить датафрей regions
regions = regions.drop(regions[([(regions.region.values[i] not in ind_list) for i in range(len(regions))])].index)

In [0]:
#Найти новые границы интересуемого района:
west  = min(regions.west)
south = min(regions.south)
north = max(regions.north)
east  = max(regions.east)

In [0]:
new_col = ['vendor_id',
'tpep_pickup_datetime',
'tpep_dropoff_datetime',
'passenger_count',
'trip_distance',
'pickup_longitude',
'pickup_latitude',
'rate_code',
'store_and_fwd_flag',
'dropoff_longitude',
'dropoff_latitude',
'payment_type',
'fare_amount',
'surcharge',
'mta_tax',
'tip_amount',
'tolls_amount',
'total_amount']

In [0]:
date = month_list[-1]
print(date)

2016-06


In [0]:
addr = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_"
link = str(addr + date + ".csv")
!wget $link
file = str("/content/yellow_tripdata_" + date + ".csv")

data = pd.read_csv(file)

--2019-09-05 18:22:51--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-06.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.160.93
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.160.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1748784180 (1.6G) [application/octet-stream]
Saving to: ‘yellow_tripdata_2016-06.csv’

yellow_tripdata_201 100%[===================>]   1.63G  16.3MB/s    in 1m 52s  

2019-09-05 18:24:44 (14.9 MB/s) - ‘yellow_tripdata_2016-06.csv’ saved [1748784180/1748784180]



In [0]:
!rm /content/ye*

In [0]:
#предобработка
for i in range(len(new_col)):
  data = data.rename(columns = {data.columns[i]: new_col[i]})

In [0]:
data.drop(data[data.tpep_pickup_datetime == data.tpep_dropoff_datetime].index , inplace=True)
data.drop(data[data.passenger_count == 0].index , inplace=True)
data.drop(data[data.trip_distance == 0].index , inplace=True)
data.drop(data[data.pickup_longitude <  west].index , inplace=True)
data.drop(data[data.pickup_longitude >  east].index , inplace=True)
data.drop(data[data.pickup_latitude  < south].index , inplace=True)
data.drop(data[data.pickup_latitude  > north].index , inplace=True)

In [0]:
#конвертация
data.tpep_pickup_datetime  = data.tpep_pickup_datetime.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [0]:
rem_col =['store_and_fwd_flag','surcharge','mta_tax','tip_amount','tolls_amount','total_amount', 'dropoff_longitude', 'dropoff_latitude', 'tpep_dropoff_datetime']

In [0]:
data = data.drop(rem_col, axis=1)

In [0]:
#категориальные признаки vendor_id, rate_code, payment_type
data['vendor_id'] = data['vendor_id'].astype('category')
data['payment_type'] = data['payment_type'].astype('category')
data['rate_code'] = data['rate_code'].astype('category')

In [0]:
data = pd.concat([data, pd.get_dummies(data.vendor_id, prefix = 'vendor')], axis = 1)

In [0]:
data.drop(['vendor_id'], axis='columns', inplace=True)

In [0]:
data = pd.concat([data, pd.get_dummies(data.payment_type, prefix = 'pay')], axis = 1)
data.drop(['payment_type'], axis='columns', inplace=True)

In [0]:
lost_columns = ['pay_1', 'pay_2', 'pay_3', 'pay_4', 'pay_5']
for lcol in lost_columns:
  if lcol not in data.columns:
    data[lcol] = 0

In [0]:
data = pd.concat([data, pd.get_dummies(data.rate_code, prefix = 'rate')], axis = 1)
data.drop(['rate_code'], axis='columns', inplace=True)

In [0]:
lost_columns = ['rate_1', 'rate_2', 'rate_3', 'rate_4', 'rate_5', 'rate_6', 'rate_99']
for lcol in lost_columns:
  if lcol not in data.columns:
    data[lcol] = 0

In [0]:
start_time = min(data.tpep_pickup_datetime)
days = calendar.monthrange(start_time.year,start_time.month)[1]
hours_cnt = days * 24

In [0]:
hour_index = [(start_time + timedelta(hours=1) * (i)) for i in range(hours_cnt)]

In [0]:
reg_col = ['passenger_count', 'trip_distance', 'fare_amount', 'vendor_1', 'vendor_2', 'pay_1', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'rate_1', 'rate_2', 'rate_3', 'rate_4', 'rate_5', 'rate_6', 'rate_99']

In [0]:
%%time
#Цикл для региона
for reg in range(len(regions)):
  curren_reg = []
  cdf = data[(data.pickup_longitude > regions.west.values[reg])
           & (data.pickup_longitude < regions.east.values[reg])
           & (data.pickup_latitude  > regions.south.values[reg])
           & (data.pickup_latitude  < regions.north.values[reg])]

  for t in range(hours_cnt):
    current_hour = cdf[(cdf.tpep_pickup_datetime >= start_time + timedelta(hours=1) * (t)) & (cdf.tpep_pickup_datetime < start_time + timedelta(hours=1) * (t + 1))]
    curren_reg.append([current_hour[reg_col[i]].mean() for i in range(len(reg_col))])
    
  regions_df = pd.DataFrame(curren_reg, columns = reg_col, index = hour_index)
  regions_df.to_csv('/content/gdrive/My Drive/ml/c6/c6w6/regions3/' + str(date) + str('_') + str(regions.region.values[reg]) + str('.csv'))  

CPU times: user 5min 49s, sys: 1.27 s, total: 5min 50s
Wall time: 5min 54s


###Склеить датасеты по регионам в папку /regions4

In [0]:
#список файлов:
path = '/content/gdrive/My Drive/ml/c6/c6w6/regions3'
fl = []
for filename in os.listdir(path):
  fl.append(filename)

In [0]:
%%time
for r in range(len(regions)):
  df_list = []
  for f in fl:
    if f[8:12] == str(regions.region.values[r]):
      df_list.append(f)
  #скачать датафреймы
  path = '/content/gdrive/My Drive/ml/c6/c6w6/regions3/'
  df = [pd.read_csv(path + f, index_col = 0) for f in df_list]
  #склеить датафреймы и сохранить
  (pd.concat(df)).to_csv('/content/gdrive/My Drive/ml/c6/c6w6/regions4/' + str(str(regions.region.values[r])) + '.csv')

CPU times: user 10 s, sys: 513 ms, total: 10.5 s
Wall time: 4min 19s


###Сгенерировать датасеты по количество поездок в текущий регион

In [0]:
%%time
def aggregator(date, xbins, ybins):
  
  #загрузка
  addr = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_"
  link = str(addr + date + ".csv")
  !wget $link
  file = str("/content/yellow_tripdata_" + date + ".csv")
  data = pd.read_csv(file)
  
  #предобработка
  data = data.rename(columns = {data.columns[1] : 'tpep_pickup_datetime'})
  data = data.rename(columns = {data.columns[2] : 'tpep_dropoff_datetime'})
  data = data.rename(columns = {data.columns[3] : 'passenger_count'})
  data = data.rename(columns = {data.columns[4] : 'trip_distance'})
  data = data.rename(columns = {data.columns[5] : 'pickup_longitude'})
  data = data.rename(columns = {data.columns[6] : 'pickup_latitude'})
  data = data.rename(columns = {data.columns[9] : 'dropoff_longitude'})
  data = data.rename(columns = {data.columns[10]: 'dropoff_latitude'})  
  
  
  
  #фильтрация
  data.drop(data[data.tpep_pickup_datetime == data.tpep_dropoff_datetime].index , inplace=True)
  data.drop(data[data.passenger_count == 0].index , inplace=True)
  data.drop(data[data.trip_distance == 0].index , inplace=True)
  data.drop(data[data.dropoff_longitude < -74.25559].index , inplace=True)
  data.drop(data[data.dropoff_longitude > -73.70001].index , inplace=True)
  data.drop(data[data.dropoff_latitude  <  40.49612].index , inplace=True)
  data.drop(data[data.dropoff_latitude  >  40.91553].index , inplace=True)
  
  #конвертация
  data.tpep_pickup_datetime  = data.tpep_pickup_datetime.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  data.tpep_dropoff_datetime = data.tpep_dropoff_datetime.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  
  start_time = min(data.tpep_pickup_datetime)
  hour_list = []
  days = calendar.monthrange(start_time.year,start_time.month)[1]
  hours_cnt = days * 24

  #цикл для указанного количества дней в месяце
  for i in range(hours_cnt):
    current_hour = data[(data.tpep_pickup_datetime >= start_time + timedelta(hours=1) * (i)) & (data.tpep_pickup_datetime < start_time + timedelta(hours=1) * (i + 1))]
    #arr = current_hour[['pickup_longitude', 'pickup_latitude']].values
    arr = current_hour[['dropoff_longitude', 'dropoff_latitude']].values
    ret = binned_statistic_2d(arr[:,0],arr[:,1],None,'count',bins=[xbins, ybins])
    hour_string = [ret.statistic[x][y] for x in range(50) for y in range(50)]
    hour_list.append(hour_string)

  #создать dataframe из массива. В качестве индекса использовать дату и час.
  regions_list = ['region_' + str(x) for x in range(1,2500 + 1)]
  hour_index = [start_time + timedelta(hours=1) * (x) for x in range(hours_cnt)]
  regions_df = pd.DataFrame(hour_list, columns = regions_list, index = hour_index)

  #экспорт CSV
  name = str(start_time.year) + '-' + str(start_time.month) + '.csv'
  path = '/content/gdrive/My Drive/ml/c6/c6w6/regions5/'
  regions_df.to_csv(path + name)
  
  !rm /content/ye*
  
  return start_time, ' : DONE'

for i in range(len(month_list)):
  aggregator(month_list[i], xbins, ybins)

--2019-09-05 18:34:41--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2016-06.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.137.54
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.137.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1748784180 (1.6G) [application/octet-stream]
Saving to: ‘yellow_tripdata_2016-06.csv’

yellow_tripdata_201 100%[===================>]   1.63G  45.4MB/s    in 38s     

2019-09-05 18:35:19 (44.0 MB/s) - ‘yellow_tripdata_2016-06.csv’ saved [1748784180/1748784180]

CPU times: user 5min 58s, sys: 6.49 s, total: 6min 5s
Wall time: 6min 46s


###Финальная склейка данных

In [0]:
path = '/content/gdrive/My Drive/ml/c6/c6w6/regions2/'
fl = []
for filename in os.listdir(path):
  fl.append(filename)
fl = fl[:-1]

In [0]:
#считать все датафреймы
%%time
df = [pd.read_csv(path + f, index_col = 0) for f in fl]

CPU times: user 6.1 s, sys: 467 ms, total: 6.57 s
Wall time: 1min 6s


In [0]:
path = '/content/gdrive/My Drive/ml/c6/c6w6/regions4/'
fl = []
for filename in os.listdir(path):
  fl.append(filename)

In [0]:
df_new = [pd.read_csv(path + f, index_col = 0) for f in fl]

In [0]:
#список регионов
regions = [f[0:4] for f in fl]

In [0]:
#загрузить даафреймы с дропофом
path = '/content/gdrive/My Drive/ml/c6/c6w6/regions5/'
fl = []
for filename in os.listdir(path):
  fl.append(filename)

df_dropoff = [pd.read_csv(path + f, index_col = 0) for f in fl]

#склеить датафреймы
df_dropoff = pd.concat(df_dropoff)

In [0]:
#переименовать столбцы
df_dropoff.columns = [df_dropoff.columns[i][7:] for i in range(len(df_dropoff.columns))]

In [0]:
rem_col = []
for c in df_dropoff.columns:
  if c not in regions:
    rem_col.append(c)

df_dropoff.drop(rem_col, axis='columns', inplace=True)    

In [0]:
%%time
#для каждого региона
for r in regions:
  #скачать данные из regions2
  df_old = pd.read_csv('/content/gdrive/My Drive/ml/c6/c6w6/regions2/' + r + '.csv', index_col = 0)
  df_new = pd.read_csv('/content/gdrive/My Drive/ml/c6/c6w6/regions4/' + r + '.csv', index_col = 0)
  #pd.concat([df1, df4], axis=1)
  df_fial =pd.concat([df_old, df_new, df_dropoff[r]], axis=1)
  #переименовать столбец
  df_fial.rename(columns = {r: 'dropoff'}, inplace=True)
  #сохранитьв в новый датафрейм
  df_fial.to_csv('/content/gdrive/My Drive/ml/c6/c6w6/regions_final/' + r + str('.csv'))  

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


CPU times: user 35.9 s, sys: 1.7 s, total: 37.6 s
Wall time: 1min 30s


###Predict

In [0]:
import os
#считать данные
filepath = '/content/gdrive/My Drive/ml/c6/c6w6/regions_final/'
fl = []
for filename in os.listdir(filepath):
  fl.append(filename)
df = []  
for i in range(len(fl)):
  df.append(pd.read_csv(filepath + fl[i], index_col = 0))

#склеить датафреймы  
data = pd.concat(df, sort=False)

In [0]:
#заменить признак weekday по OneHot
data = pd.concat([data, pd.get_dummies(data.weekday, prefix = 'weekday')], axis = 1)
#удалить столбец "weekday"
data.drop(['weekday'], axis='columns', inplace=True)

In [0]:
#заменить признак hour по OneHot
data = pd.concat([data, pd.get_dummies(data.hour, prefix = 'hour')], axis = 1)
#удалить столбец "hour"
data.drop(['hour'], axis='columns', inplace=True)

In [0]:
#заменить признак region по OneHot
data = pd.concat([data, pd.get_dummies(data.region, prefix = 'reg')], axis = 1)
#удалить столбец "region"
#data.drop(['region'], axis='columns', inplace=True)

In [0]:
#целевые метки
target_labels = []
for i in range(1, 7):
  target_labels.append(str('yT+' + str(i)))

In [0]:
#Обучающая выборка: train_data
data_train = data[data.index < '2016-05-01 00:00:00'].dropna()
#Тестовая выборка : test_data
data_test = data[(data.index < '2016-06-01 00:00:00') & (data.index > '2016-04-30 23:00:00')].dropna()

In [0]:
new_features = [
                'passenger_count',
                'trip_distance',
                'fare_amount',
                'vendor_1', 'vendor_2',
                'pay_1', 'pay_2', 'pay_3', 'pay_4', 'pay_5',
                'rate_1', 'rate_2', 'rate_3','rate_4', 'rate_5','rate_6', 'rate_99',
                'dropoff'
               ]

###Добавляем признаки, обучаем, считаем скор на мае

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
itr_arr = [0, 1, 2, 3, 5, 10, 17, 18]
for itr in itr_arr:
  drop_features = new_features[itr:]

  X_train = data_train.copy()
  X_test  = data_test.copy()
  X_train.drop(['region'], axis='columns', inplace=True)
  X_test.drop(['region'], axis='columns', inplace=True)

  X_train.drop(target_labels + drop_features, axis='columns', inplace=True)
  X_test.drop(target_labels + drop_features, axis='columns', inplace=True)

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)

  from sklearn.linear_model import Lasso

  alph = [0.5, 0.5, 0.5, 0.3, 0.1, 0.1]

  lasso = []
  err   = []
  regr  = []


  for i in range(6):
    y_train = pd.DataFrame(data_train[target_labels[i]]).values
    y_test  = pd.DataFrame(data_test[target_labels[i]]).values

    lasso.append(Lasso(alpha = alph))
    lasso[i] = Lasso(alpha=alph[i])
    lasso[i].fit(X_train_scaled, y_train)
    y_test_pred = lasso[i].predict(scaler.transform(X_test))
    y_test_pred = y_test_pred * (y_test_pred > 0)
    current_err = sum(abs(data_test[target_labels[i]].values - y_test_pred))/len(y_test_pred)
    print(i, alph[i], current_err)
    err.append(current_err)
    regr.append(lasso[i])

  #Ошибка за май
  print(itr, sum(err)/6)

* (0, 0.5, 22.20752123776375)
* (1, 0.5, 30.179381286743908)
* (2, 0.5, 33.927392128975534)
* (3, 0.3, 35.44096326445744)
* (4, 0.1, 36.006910952905585)
* (5, 0.1, 36.53423356833633)
### 32.38273373986376
* (0, 0.5, 22.207521240649452)
* (1, 0.5, 30.179381294578494)
* (2, 0.5, 33.92720663358572)
* (3, 0.3, 35.440875693630794)
* (4, 0.1, 36.01002511783)
* (5, 0.1, 36.53601287659417)
###32.38350380947811
* (0, 0.5, 22.207521240649466)
* (1, 0.5, 30.179381294578494)
* (2, 0.5, 33.927206633585726)
* (3, 0.3, 35.44087569363079)
* (4, 0.1, 36.010025142069956)
* (5, 0.1, 36.53601287663486)
###32.38350381352488
* (0, 0.5, 22.2075212484827)
* (1, 0.5, 30.179381255213016)
* (2, 0.5, 33.9272066309653)
* (3, 0.3, 35.440875740691105)
* (4, 0.1, 36.00979601604099)
* (5, 0.1, 36.536012955766296)
###32.38346564119323
* (0, 0.5, 22.207521248482703)
* (1, 0.5, 30.179381256826172)
* (2, 0.5, 33.92720669163874)
* (3, 0.3, 35.44108627157133)
* (4, 0.1, 36.00978320647024)
* (5, 0.1, 36.536054375282774)
###32.383505508378654
* (0, 0.5, 22.206410359101774)
* (1, 0.5, 30.179381314718317)
* (2, 0.5, 33.92720741574701)
* (3, 0.3, 35.44469353303691)
* (4, 0.1, 36.01973567998581)
* (5, 0.1, 36.538655311141774)
###32.38601393562194
* (0, 0.5, 22.206410360742726)
* (1, 0.5, 30.179381308053898)
* (2, 0.5, 33.91825383558736)
* (3, 0.3, 35.43402014066319)
* (4, 0.1, 36.018495977933675)
* (5, 0.1, 36.53827372415635)
###32.3824725578562
* (0, 0.5, 22.07438650559728)
* (1, 0.5, 30.02764998903104)
* (2, 0.5, 33.7918194231054)
* (3, 0.3, 35.3402520305047)
* (4, 0.1, 35.992304567372386)
* (5, 0.1, 36.60797821876241)
###32.30573178906221

###Ошибка практически не уменьшается с введением новых признаков

In [0]:
for i in range(len(regr)):
  print(zip(X_test.columns[regr[i].coef_ > 0], regr[i].coef_[regr[i].coef_ > 0]))

Наибольший вклад вносит регрессор, содержащий о количестве поездок в текущий регион

###Попробуем использовать регрессор **Ridge**

In [0]:
from sklearn.linear_model import Ridge

itr_arr = [0, 1, 2, 3, 5, 10, 17, 18]

for itr in itr_arr:
  drop_features = new_features[itr:]

  X_train = data_train.copy()
  X_test  = data_test.copy()
  
  X_train.drop(['region'], axis='columns', inplace=True)
  X_test.drop(['region'], axis='columns', inplace=True)  

  X_train.drop(target_labels + drop_features, axis='columns', inplace=True)
  X_test.drop(target_labels + drop_features, axis='columns', inplace=True)

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)

  alph = [0.5, 0.5, 0.5, 0.3, 0.1, 0.1]

  ridge = []
  err   = []
  regr  = []


  for i in range(6):
    y_train = pd.DataFrame(data_train[target_labels[i]]).values
    y_test  = pd.DataFrame(data_test[target_labels[i]]).values

    ridge.append(Ridge(alpha = alph[i]))
    ridge[i].fit(X_train_scaled, y_train)
    y_test_pred = ridge[i].predict(scaler.transform(X_test))
    y_test_pred = y_test_pred * (y_test_pred > 0)
    y_test_pred = np.asarray([float(y_test_pred[j]) for j in range(len(y_test_pred))])
    data_test_values = np.asarray([float(data_test[target_labels[i]].values[j]) for j in range(len(data_test[target_labels[i]].values))])
    current_err = sum(abs(data_test_values - y_test_pred))/len(y_test_pred)
    print(i, alph[i], current_err)
    err.append(current_err)
    regr.append(ridge[i])

  #Ошибка за май
  print(itr, sum(err)/6)

* (0, 0.5, 22.541722796727065)
* (1, 0.5, 30.863862601844488)
* (2, 0.5, 34.494453326649435)
* (3, 0.3, 35.883274685275794)
* (4, 0.1, 36.30808570529876)
* (5, 0.1, 36.81249721255687)
###32.8173160547254
* (0, 0.5, 22.54100200789687)
* (1, 0.5, 30.864202886959717)
* (2, 0.5, 34.494076699000715)
* (3, 0.3, 35.882224513219604)
* (4, 0.1, 36.3107269593109)
* (5, 0.1, 36.81408017748324)
###32.81771887397851
* (0, 0.5, 22.54254368694377)
* (1, 0.5, 30.864186114806273)
* (2, 0.5, 34.493481122047804)
* (3, 0.3, 35.88313161558779)
* (4, 0.1, 36.315327035611766)
* (5, 0.1, 36.813479375598774)
###32.81869149176603
* (0, 0.5, 22.543127799709268)
* (1, 0.5, 30.864645650979302)
* (2, 0.5, 34.493189445500406)
* (3, 0.3, 35.88163341597121)
* (4, 0.1, 36.31408302692054)
* (5, 0.1, 36.813450885357376)
###32.81835503740635
* (0, 0.5, 22.543529144083667)
* (1, 0.5, 30.86534012255069)
* (2, 0.5, 34.49305181636529)
* (3, 0.3, 35.88031191188721)
* (4, 0.1, 36.312013520954714)
* (5, 0.1, 36.81285537170404)
###32.817850314590935
* (0, 0.5, 22.536251887332302)
* (1, 0.5, 30.858452259260694)
* (2, 0.5, 34.49138933558811)
* (3, 0.3, 35.88521733666319)
* (4, 0.1, 36.32212119425057)
* (5, 0.1, 36.816833920177714)
###32.818377655545426
* (0, 0.5, 22.53728733063502)
* (1, 0.5, 30.85523236252718)
* (2, 0.5, 34.483208698138704)
* (3, 0.3, 35.88423245795009)
* (4, 0.1, 36.324699341579446)
* (5, 0.1, 36.81805157458451)
###32.81711862756916
* (0, 0.5, 22.383586925520753)
* (1, 0.5, 30.65321815963247)
* (2, 0.5, 34.25857315314474)
* (3, 0.3, 35.72132789185394)
* (4, 0.1, 36.23607878963575)
* (5, 0.1, 36.824675892425276)
###32.679576802035484


Наибольший вклад вносит регрессор, содержащий о количестве поездок в текущий регион. Но улучшение составляет всего 0.4%

###Попробуем использовтаь в качестве регрессора RandomForestRegressor

In [0]:
from sklearn.ensemble import RandomForestRegressor

itr_arr = [0, 1, 2, 3, 5, 10, 17, 18]

for itr in itr_arr:
  drop_features = new_features[itr:]

  X_train = data_train.copy()
  X_test  = data_test.copy()
  
  X_train.drop(['region'], axis='columns', inplace=True)
  X_test.drop(['region'], axis='columns', inplace=True)  

  X_train.drop(target_labels + drop_features, axis='columns', inplace=True)
  X_test.drop(target_labels + drop_features, axis='columns', inplace=True)

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)

  forest = []
  err   = []
  regr  = []

  for i in range(6):
    y_train = pd.DataFrame(data_train[target_labels[i]]).values
    y_test  = pd.DataFrame(data_test[target_labels[i]]).values

    forest.append(RandomForestRegressor(random_state=42, n_estimators=10))
    forest[i].fit(X_train_scaled, y_train)
    y_test_pred = forest[i].predict(scaler.transform(X_test))
    y_test_pred = y_test_pred * (y_test_pred > 0)
    y_test_pred = np.asarray([float(y_test_pred[j]) for j in range(len(y_test_pred))])
    data_test_values = np.asarray([float(data_test[target_labels[i]].values[j]) for j in range(len(data_test[target_labels[i]].values))])
    current_err = sum(abs(data_test_values - y_test_pred))/len(y_test_pred)
    print(i, current_err)
    err.append(current_err)
    regr.append(forest[i])

  #Ошибка за май
  print(itr, sum(err)/6)


In [0]:
#itr_arr = [0, 1, 2, 3, 5, 10, 17, 18]
itr_arr = [0, 18]

for itr in itr_arr:
  drop_features = new_features[itr:]

  X_train = data_train.copy()
  X_test  = data_test.copy()

  X_train.drop(target_labels + drop_features, axis='columns', inplace=True)
  X_test.drop(target_labels + drop_features, axis='columns', inplace=True)

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)

  forest = []
  err   = []
  regr  = []
  for i in range(6):
    y_train = pd.DataFrame(data_train[target_labels[i]]).values
    y_test  = pd.DataFrame(data_test[target_labels[i]]).values
    forest.append(RandomForestRegressor(max_features = len(list(X_train.columns)),random_state=42, n_estimators=10))
    forest[i].fit(X_train_scaled, y_train)
    y_test_pred = forest[i].predict(scaler.transform(X_test))
    y_test_pred = y_test_pred * (y_test_pred > 0)
    y_test_pred = np.asarray([float(y_test_pred[j]) for j in range(len(y_test_pred))])
    data_test_values = np.asarray([float(data_test[target_labels[i]].values[j]) for j in range(len(data_test[target_labels[i]].values))])
    current_err = sum(abs(data_test_values - y_test_pred))/len(y_test_pred)
    print(len(list(X_train.columns)), current_err)
    err.append(current_err)
    regr.append(forest[i])
  
  #Ошибка за май
  print(itr, sum(err)/6)

* (193, 16.995036563442227)
* (193, 20.164826104000007)
* (193, 21.549111258296367)
* (193, 21.84710263446021)
* (193, 21.98956759167908)
* (193, 22.098855111447957)
####20.77408321055431

* (211, 16.92275313931969)
* (211, 19.940190048796467)
* (211, 21.241518768332575)
* (211, 21.533181492545864)
* (211, 21.814652410754547)
* (211, 22.05540611778692)
####20.58461699625601

###Добавление новых признаков не привело к существенному улучшению результата.

###Улучшение результата было получено путем применения вместо Lasso RandomForestRegressor.

###Попробуем использовать в качестве признака предсказание линейного регрессора.

In [0]:
import xgboost

In [0]:
X_train = data_train.copy()
X_test  = data_test.copy()

X_train.drop(target_labels, axis='columns', inplace=True)
X_test.drop(target_labels, axis='columns', inplace=True)

X_train.drop(['region'], axis='columns', inplace=True)
X_test.drop(['region'], axis='columns', inplace=True)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

forest = []
err   = []
regr  = []

for i in range(6):
  y_train = pd.DataFrame(data_train[target_labels[i]]).values
  y_test  = pd.DataFrame(data_test[target_labels[i]]).values

  forest.append(RandomForestRegressor(random_state=42, n_estimators=10))
  forest[i].fit(X_train_scaled, y_train)
  
  y_train_pred = forest[i].predict(X_train_scaled)
  
  y_test_pred = forest[i].predict(scaler.transform(X_test))
  #y_test_pred = y_test_pred * (y_test_pred > 0)
  y_test_pred = np.asarray([float(y_test_pred[j]) for j in range(len(y_test_pred))])
  
  xgb = xgboost.XGBRegressor()
  
  xgb.fit(y_train_pred.reshape(-1, 1),y_train)
  
  y_test_pred_xgb = xgb.predict(y_test_pred.reshape(-1, 1))
  y_test_pred_xgb = np.asarray([float(y_test_pred_xgb[j]) for j in range(len(y_test_pred_xgb))])
    
  data_test_values = np.asarray([float(data_test[target_labels[i]].values[j]) for j in range(len(data_test[target_labels[i]].values))])
  
  current_err = sum(abs(data_test_values - y_test_pred))/len(y_test_pred)
  
  current_err_xgb = sum(abs(data_test_values - (y_test_pred_xgb)))//len(y_test_pred_xgb)
  
  print(current_err, current_err_xgb)
  
#Ошибка за май
print(sum(err)/6)

* (16.92275313931969, 16.0)
* (19.940190048796467, 19.0)
* (21.241518768332575, 21.0)
* (21.533181492545864, 21.0)
* (21.814652410754547, 21.0)
* (22.05540611778692, 22.0)

Ошибка практически не отличается

###Использовать просто XGBRegressor

https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [0]:
X_train = data_train.copy()
X_test  = data_test.copy()

X_train.drop(target_labels, axis='columns', inplace=True)
X_test.drop(target_labels, axis='columns', inplace=True)

X_train.drop(['region'], axis='columns', inplace=True)
X_test.drop(['region'], axis='columns', inplace=True)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

xgb  = []
err  = []
regr = []

for i in range(6):
  y_train = pd.DataFrame(data_train[target_labels[i]]).values
  y_test  = pd.DataFrame(data_test[target_labels[i]]).values

  xgb.append(xgboost.XGBRegressor())
  xgb[i].fit(X_train_scaled, y_train)
  
  y_train_pred = xgb[i].predict(X_train_scaled)
  
  y_test_pred = xgb[i].predict(scaler.transform(X_test))
  y_test_pred = y_test_pred * (y_test_pred > 0)
  y_test_pred = np.asarray([float(y_test_pred[j]) for j in range(len(y_test_pred))])
  
  data_test_values = np.asarray([float(data_test[target_labels[i]].values[j]) for j in range(len(data_test[target_labels[i]].values))])
  
  current_err = sum(abs(data_test_values - y_test_pred))/len(y_test_pred)
  err.append(current_err)
  
  print(i, current_err)

###Подобрать параметры регрессора xgboost.XGBRegressor:
* max_depth = [5, 10 ,15, 20]
* n_estimators = [10, 20, 30, 40]
* booster='gbtree'

In [0]:
X_train = data_train.copy()
X_test  = data_test.copy()

X_train.drop(target_labels, axis='columns', inplace=True)
X_test.drop(target_labels, axis='columns', inplace=True)

X_train.drop(['region'], axis='columns', inplace=True)
X_test.drop(['region'], axis='columns', inplace=True)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

xgb  = []
err  = []
regr = []

dpth = [5, 10 ,15, 20]
etmr = [10, 20, 30, 40]

for i in range(1):
  xgb.append(xgboost.XGBRegressor())
  for dp in dpth:
    for et in etmr:
      y_train = pd.DataFrame(data_train[target_labels[i]]).values
      y_test  = pd.DataFrame(data_test[target_labels[i]]).values

      xgb[i] = xgboost.XGBRegressor(max_depth = dp, n_estimators = et, booster='gbtree')
      xgb[i].fit(X_train_scaled, y_train)

      y_train_pred = xgb[i].predict(X_train_scaled)

      y_test_pred = xgb[i].predict(scaler.transform(X_test))
      y_test_pred = y_test_pred * (y_test_pred > 0)
      y_test_pred = np.asarray([float(y_test_pred[j]) for j in range(len(y_test_pred))])

      data_test_values = np.asarray([float(data_test[target_labels[i]].values[j]) for j in range(len(data_test[target_labels[i]].values))])

      current_err = sum(abs(data_test_values - y_test_pred))/len(y_test_pred)
      err.append(current_err)

      print(dp, et, current_err)

* (5, 20, 26.22557845281437)
* (5, 30, 20.15572813177155)
* (5, 40, 18.876282675187454)
* (10, 10, 54.82483995815123)
* (10, 20, 23.64291181584602)
* (10, 30, 16.827937215589596)
* **(10, 40, 15.69156572822293)**
* (15, 10, 55.357028210757385)
* (15, 20, 24.276919002762156)
* (15, 30, 17.035181674901395)
* (15, 40, 15.708313111454057)
* (20, 10, 55.75038714376269)
* (20, 20, 24.97764331887582)
* (20, 30, 17.603414529251086)
* (20, 40, 16.091377558952924)

best_param: (10, 40, 15.69156572822293)

In [0]:
import xgboost

X_train = data_train.copy()
X_test  = data_test.copy()

X_train.drop(target_labels, axis='columns', inplace=True)
X_test.drop(target_labels, axis='columns', inplace=True)

X_train.drop(['region'], axis='columns', inplace=True)
X_test.drop(['region'], axis='columns', inplace=True)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

xgb  = []
err  = []
regr = []

for i in range(6):
  xgb.append(xgboost.XGBRegressor())
  y_train = pd.DataFrame(data_train[target_labels[i]]).values
  y_test  = pd.DataFrame(data_test[target_labels[i]]).values
  xgb[i] = xgboost.XGBRegressor(max_depth = 10, n_estimators = 40, booster='gbtree')
  xgb[i].fit(X_train_scaled, y_train)
  y_train_pred = xgb[i].predict(X_train_scaled)
  y_test_pred = xgb[i].predict(scaler.transform(X_test))
  y_test_pred = y_test_pred * (y_test_pred > 0)
  y_test_pred = np.asarray([float(y_test_pred[j]) for j in range(len(y_test_pred))])
  data_test_values = np.asarray([float(data_test[target_labels[i]].values[j]) for j in range(len(data_test[target_labels[i]].values))])
  current_err = sum(abs(data_test_values - y_test_pred))/len(y_test_pred)
  err.append(current_err)
  print(i, current_err)

#Ошибка за май
print(sum(err)/6)

* (0, 15.69156572822293)
* (1, 18.648858725871953)
* (2, 19.898023084097087)
* (3, 20.550921661327024)
* (4, 20.879636177938142)
* (5, 21.12582665576215)

**19.46580533886988**

###Переобучить модель на данных по май включительно

###Сгенерировать почасовой прогноз на июнь 2016

In [0]:
#данные для обучения
data_train = data[data.index < '2016-06-01 00:00:00'].dropna()
data_test = data[(data.index >= '2016-05-31 23:00:00') & (data.index < '2016-06-30 18:00:00')]

In [0]:
data_test.fillna(0)

,yT,yT-1,yT-2,yT-3,yT-4,yT-5,yT-6,yT-7,yT-8,yT-9,yT-10,yT-11,yT-12,yT-13,yT-14,yT-15,yT-16,yT-17,yT-18,yT-19,yT-20,yT-21,yT-22,yT-23,yT-24,yT-48,yT-72,yT-96,yT-120,yT-144,yT-168,sum_12,sum_24,sum_168,sum_672,region,sin_1,cos_1,sin_2,cos_2,...,reg_1376,reg_1377,reg_1378,reg_1380,reg_1382,reg_1383,reg_1384,reg_1385,reg_1386,reg_1387,reg_1388,reg_1389,reg_1390,reg_1426,reg_1431,reg_1434,reg_1435,reg_1436,reg_1437,reg_1438,reg_1439,reg_1441,reg_1442,reg_1480,reg_1482,reg_1483,reg_1530,reg_1532,reg_1533,reg_1580,reg_1630,reg_1684,reg_1733,reg_1734,reg_1783,reg_2068,reg_2069,reg_2118,reg_2119,reg_2168
2016-05-31 23:00:00,341.0,449.0,452.0,424.0,418.0,463.0,396.0,318.0,398.0,389.0,362.0,325.0,352.0,445.0,462.0,603.0,524.0,341.0,121.0,69.0,55.0,65.0,112.0,162.0,394.0,440.0,576.0,573.0,669.0,620.0,495.0,4746.0,8099.0,60665.0,283184.0,1229,-0.974928,-2.225209e-01,4.338837e-01,-9.009689e-01,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 00:00:00,229.0,341.0,449.0,452.0,424.0,418.0,463.0,396.0,318.0,398.0,389.0,362.0,325.0,352.0,445.0,462.0,603.0,524.0,341.0,121.0,69.0,55.0,65.0,112.0,162.0,348.0,484.0,526.0,522.0,369.0,298.0,4735.0,8046.0,60511.0,283043.0,1229,-0.982566,-1.859116e-01,3.653410e-01,-9.308737e-01,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 01:00:00,149.0,229.0,341.0,449.0,452.0,424.0,418.0,463.0,396.0,318.0,398.0,389.0,362.0,325.0,352.0,445.0,462.0,603.0,524.0,341.0,121.0,69.0,55.0,65.0,112.0,313.0,452.0,464.0,390.0,251.0,171.0,4639.0,8113.0,60442.0,282977.0,1229,-0.988831,-1.490423e-01,2.947552e-01,-9.555728e-01,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 02:00:00,67.0,149.0,229.0,341.0,449.0,452.0,424.0,418.0,463.0,396.0,318.0,398.0,389.0,362.0,325.0,352.0,445.0,462.0,603.0,524.0,341.0,121.0,69.0,55.0,65.0,201.0,368.0,390.0,219.0,136.0,108.0,4426.0,8150.0,60420.0,282922.0,1229,-0.993712,-1.119645e-01,2.225209e-01,-9.749279e-01,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 03:00:00,51.0,67.0,149.0,229.0,341.0,449.0,452.0,424.0,418.0,463.0,396.0,318.0,398.0,389.0,362.0,325.0,352.0,445.0,462.0,603.0,524.0,341.0,121.0,69.0,55.0,158.0,277.0,331.0,125.0,100.0,63.0,4104.0,8152.0,60379.0,282886.0,1229,-0.997204,-7.473009e-02,1.490423e-01,-9.888308e-01,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 04:00:00,48.0,51.0,67.0,149.0,229.0,341.0,449.0,452.0,424.0,418.0,463.0,396.0,318.0,398.0,389.0,362.0,325.0,352.0,445.0,462.0,603.0,524.0,341.0,121.0,69.0,125.0,221.0,217.0,117.0,77.0,60.0,3757.0,8148.0,60367.0,282875.0,1229,-0.999301,-3.739119e-02,7.473009e-02,-9.972038e-01,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 05:00:00,125.0,48.0,51.0,67.0,149.0,229.0,341.0,449.0,452.0,424.0,418.0,463.0,396.0,318.0,398.0,389.0,362.0,325.0,352.0,445.0,462.0,603.0,524.0,341.0,121.0,73.0,71.0,104.0,155.0,139.0,105.0,3487.0,8127.0,60355.0,282855.0,1229,-1.000000,-1.420900e-14,2.841800e-14,-1.000000e+00,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 06:00:00,348.0,125.0,48.0,51.0,67.0,149.0,229.0,341.0,449.0,452.0,424.0,418.0,463.0,396.0,318.0,398.0,389.0,362.0,325.0,352.0,445.0,462.0,603.0,524.0,341.0,100.0,80.0,143.0,342.0,376.0,367.0,3216.0,8131.0,60375.0,282883.0,1229,-0.999301,3.739119e-02,-7.473009e-02,-9.972038e-01,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 07:00:00,572.0,348.0,125.0,48.0,51.0,67.0,149.0,229.0,341.0,449.0,452.0,424.0,418.0,463.0,396.0,318.0,398.0,389.0,362.0,325.0,352.0,445.0,462.0,603.0,524.0,114.0,117.0,181.0,555.0,552.0,587.0,3101.0,8138.0,60356.0,282855.0,1229,-0.997204,7.473009e-02,-1.490423e-01,-9.888308e-01,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-01 08:00:00,577.0,572.0,348.0,12

In [0]:
len(data_test)

72930

In [0]:
import xgboost

In [0]:
X_train = data_train.copy()
X_test  = data_test.copy()

X_train.drop(target_labels, axis='columns', inplace=True)
X_test.drop(target_labels, axis='columns', inplace=True)

X_train.drop(['region'], axis='columns', inplace=True)
X_test.drop(['region'], axis='columns', inplace=True)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

predict = []

xgb = xgboost.XGBRegressor(max_depth = 10, n_estimators = 40, booster='gbtree')

for i in range(6):
  y_train = pd.DataFrame(data_train[target_labels[i]]).values
  xgb.fit(X_train_scaled, y_train)
  y_test_pred = xgb.predict(scaler.transform(X_test))
  y_test_pred = y_test_pred * (y_test_pred > 0)
  y_test_pred = np.asarray([float(y_test_pred[j]) for j in range(len(y_test_pred))])
  predict.append(y_test_pred)
  print(str('yT+') + str(i + 1) + str(': DONE'))

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python2.7/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.


yT+1: DONE
yT+2: DONE
yT+3: DONE
yT+4: DONE
yT+5: DONE
yT+6: DONE


In [0]:
df_list = []
id_list = []
for i in range((len(target_labels))):
  df = pd.DataFrame(columns = ['id', 'y'])
  id_list.append(list(map(lambda x, y : str(x) + str('_') + str(y)[:10] + str('_') + str(int(str(y)[11:13])) + str('_') + str(i + 1), data_test.region, data_test.index)))
  df.id = id_list[i]
  df.y = predict[i]
  df_list.append(df)

In [0]:
data_final = pd.concat(df_list)

In [0]:
data_final.to_csv('/content/gdrive/My Drive/ml/c6/c6w6/final_c6w6_v2.csv', index=False)

In [0]:
data_final.head()

,id,y
0,1229_2016-05-31_23_1,234.895355
1,1229_2016-06-01_0_1,131.861801
2,1229_2016-06-01_1_1,92.819893
3,1229_2016-06-01_2_1,80.366158
4,1229_2016-06-01_3_1,76.110291


In [0]:
len(data_final)

437580

###Загрузить данные на https://inclass.kaggle.com/c/yellowtaxi

116
Konstantin Shitkov
Konstantin Shitkovnov
24.73073
https://inclass.kaggle.com/submissions/12399268/12399268.raw